## **Repositorio de GitHub**

### Caso de negocio

Los objetivos del presente proyecto son la siguientes:

1. Conocer los perfiles de riesgo de los conductores asegurados en relación a sufrir accidentes de tráfico, con el fin de ajustar las cuantías de las primas de seguro exigidas a dichos usuarios.
2. Intentar predecir en qué casos se produce un accidente mortal, para poder llevar a cabo una inmovilización de tesorería lo más ajustada posible a los potenciales requerimientos de caja futuros derivados del fallecimiento del asegurado. Todo ello para reducir los recursos ociosos de la compañía.

Se busca la consecución de estas dos metas por medio del estudio de características del vehículo, conductor/persona y condiciones propias del accidente, en el instante anterior de producirse el evento y cronológicamente disponibles hasta esta el instante de iniciarse el análisis.

### Funciones (GUARDAR EN UN ARCHIVO Pickle UNA VEZ ACABADO EL TRABAJO E IMPORTADO COMO OTRO LIBRERÍA MÁS)

In [13]:
def auto_report(xtest, ytest, ypred):
    """Toma datasets de prueba y predicción para hacer un informe con las principales medidas de evaluación del modelo

    Args:
        xtest ([pandas.DataFrame]): Dataset de validación
        ytest ([np.array]): Valores de la variable dependiente para validación
        ypred ([np.array]): Valores estimados por el modelo
    """
    print(clf.score(xtest, ytest))
    # classification_metrics = classification_report(ytest, ypred)
    print('\n',classification_report(ytest, ypred))
    # f1_score = f1_score(ytest, ypred)
    print("\nF1:", f1_score(ytest, ypred))

In [11]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

### Importación de librerías y datos

Importación de las librerías y funciones necesarias:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings

Desactivación de avisos y especifiación de dimensiones máximas visualizables de los dataframes:

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Importación de datos desde dirección relativa:

In [3]:
df_original = pd.read_csv("../data/car_accidents_canada.csv")

Número de registros y variables contenidas por el modelo:

In [7]:
df_original.shape

(5860405, 22)

Resumen del tipo de datos contenidos en cada variable/columna:

In [4]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5860405 entries, 0 to 5860404
Data columns (total 22 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   C_YEAR  int64 
 1   C_MNTH  object
 2   C_WDAY  object
 3   C_HOUR  object
 4   C_SEV   int64 
 5   C_VEHS  object
 6   C_CONF  object
 7   C_RCFG  object
 8   C_WTHR  object
 9   C_RSUR  object
 10  C_RALN  object
 11  C_TRAF  object
 12  V_ID    object
 13  V_TYPE  object
 14  V_YEAR  object
 15  P_ID    object
 16  P_SEX   object
 17  P_AGE   object
 18  P_PSN   object
 19  P_ISEV  object
 20  P_SAFE  object
 21  P_USER  object
dtypes: int64(2), object(20)
memory usage: 983.6+ MB


Vistazo de los 5 primeros registros/filas del dataframe:

In [8]:
df_original.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,1,1,20,2,02,34,UU,1,5,3,03,01,06,1990,01,M,41,11,1,UU,1
1,1999,1,1,20,2,02,34,UU,1,5,3,03,02,01,1987,01,M,19,11,1,UU,1
2,1999,1,1,20,2,02,34,UU,1,5,3,03,02,01,1987,02,F,20,13,2,02,2
3,1999,1,1,08,2,01,01,UU,5,3,6,18,01,01,1986,01,M,46,11,1,UU,1
4,1999,1,1,08,2,01,01,UU,5,3,6,18,99,NN,NNNN,01,M,05,99,2,UU,3


**Una vez cargados los datos, se procede a su limpieza y estudio descriptivo (Wrangling + EDA)**

## **Análisis exploratorio-descriptivo (EDA)**

Para no porder datos que se proporcionan inicialmente y buscando discriminar la información menos relevante, se ha creado un índice doble que contiene tanto la información sobre la matrícula del vehículo como del ID del conductor:

In [9]:
df_original.set_index(['P_ID', 'V_ID'])

C_YEAR C_MNTH C_WDAY C_HOUR  C_SEV C_VEHS C_CONF C_RCFG C_WTHR  \
P_ID V_ID                                                                   
01   01      1999      1      1     20      2     02     34     UU      1   
     02      1999      1      1     20      2     02     34     UU      1   
02   02      1999      1      1     20      2     02     34     UU      1   
01   01      1999      1      1     08      2     01     01     UU      5   
     99      1999      1      1     08      2     01     01     UU      5   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
     13      2014     UU      U     UU      2     UU     UU     01      U   
     1       2014     UU      U     23      2     01     06     05      1   
     1       2014     UU      U     14      2     01     02     01      1   
     1       2014     UU      U     22      1     01     06     01      2   
02   1       2014     UU      U     22      1     01     06     01      2   

          C_RSUR C_RALN C_TRAF V_TYPE V_YEAR P_SEX P_AGE P_PSN P_ISEV P_SAFE  \
P_ID V_ID                                                                      
01   01        5      3     03     06   1990     M    41    11      1     UU   
     02        5      3     03     01   1987     M    19    11      1     UU   
02   02        5      3     03     01   1987     F    20    13      2     02   
01   01        3      6     18     01   1986     M    46    11      1     UU   
     99        3      6     18     NN   NNNN     M    05    99      2     UU   
...          ...    ...    ...    ...    ...   ...   ...   ...    ...    ...   
     13        U      U     UU     07   UUUU     M    24    11      1     NN   
     1         1      U     18     14   2006     M    29    96      2     09   
     1         5      4     18     01   2006     F    UU    11      2     01   
     1         4      Q     18     22   UUUU     M    67    12      3     01   
02   1         4      Q     18     22   UUUU     M    10    98      1     01   

          P_USER  
P_ID V_ID         
01   01        1  
     02        1  
02   02        2  
01   01        1  
     99        3  
...          ...  
     13        1  
     1         5  
     1         1  
     1         U  
02   1         U  

[5860405 rows x 20 columns]